VERİLEN SES KAYITLARININ MFCC ÖZELLİKLERİNİN ÇIKARILMASI 

In [5]:
import librosa
import numpy as np
import os

# Ses dosyalarınızı kaydettiğiniz klasör
ses_dosyasi_klasoru = "ses_verileri"

def ozellik_cikarma(file_path):
    # Ses dosyasını yükleme
    y, sr = librosa.load(file_path, duration=5)  # 5 saniye sınırlaması
    # MFCC özelliklerini çıkarma
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    # Ortalama alarak sabit boyutlu bir vektör elde etme
    return np.mean(mfcc, axis=1)

# Tüm dosyalar için özellik çıkarma
veri = []
etiketler = []

for file in os.listdir(ses_dosyasi_klasoru):
    if file.endswith('.wav'):
        file_path = os.path.join(ses_dosyasi_klasoru, file)
        features = ozellik_cikarma(file_path)
        veri.append(features)
        etiketler.append(file.split('.')[0])  # Dosya adını etiket olarak kullanma

veri = np.array(veri)
etiketler = np.array(etiketler)


MODELİ EĞİTME KAYDI VERİLEN KİSİNİN KİM OLDUGUNU BULMA

In [12]:
import librosa 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
import os

# Özellik çıkarma fonksiyonu
def ozellik_cikarma(file_path):
    y, sr = librosa.load(file_path, duration=5)  # Ses dosyasını yükleme
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # MFCC çıkarma
    return np.mean(mfcc, axis=1)  # Ortalama ile sabit boyutlu vektör

# Veriyi yükleme
def veri_yukle(ses_klasoru):
    veri = []
    etiketler = []
    for file in os.listdir(ses_klasoru):
        if file.endswith('.wav'):
            file_path = os.path.join(ses_klasoru, file)
            features = ozellik_cikarma(file_path)
            veri.append(features)
            etiketler.append(file.split('_')[0])  # Kişi adı dosya adından alınır
    return np.array(veri), np.array(etiketler)

# Ses dosyalarının bulunduğu klasör
ses_klasoru = "ses_verileri"
X, y = veri_yukle(ses_klasoru)

# Etiketleri sayısal forma dönüştürme
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Veriyi eğitim ve test olarak ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.25, random_state=42)

# Veri türlerini kontrol etme ve düzeltme
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Model oluşturma
model = Sequential([
    Input(shape=(X_train.shape[1],)),  # Giriş: Özellik sayısı
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')  # Çıkış: Kişi sayısı
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Modeli eğitme
model.fit(X_train, y_train, epochs=20, batch_size=4, validation_data=(X_test, y_test), verbose=0)

# Modeli kaydetme
model.save("ses_tanima_modeli.keras")
print("Model başarıyla eğitildi ve kaydedildi.")

# Yeni ses dosyasını tahmin etme
def ses_tahmin_et(model_path, file_path, label_encoder):
    model = load_model(model_path , compile=False)
    features = ozellik_cikarma(file_path).reshape(1, -1)  # Model girişine uygun boyut
    tahmin = model.predict(features)
    tahmin_kisi = label_encoder.inverse_transform([np.argmax(tahmin)])
    return tahmin_kisi[0]

# Modelin doğruluğunu ve F1 skorunu değerlendirme
y_pred = model.predict(X_test)  # Test verisi üzerindeki tahminler
y_pred_class = np.argmax(y_pred, axis=1)  # Sınıf tahminlerini al (softmax sonucu)

# F1 skoru hesaplama
f1 = f1_score(y_test, y_pred_class, average='weighted')  # weighted: Sınıf dengesizliğini göz önünde bulundurur

# Test için bir ses dosyası tahmini
test_ses_dosyasi = "ses_verileri/aysu_01.wav"  # Test edilecek dosya
tahmin = ses_tahmin_et("ses_tanima_modeli.keras", test_ses_dosyasi, le)
print(f"Tahmin edilen kişi: {tahmin}")

# Modelin doğruluğunu değerlendirme
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Test seti üzerindeki doğruluk(accuracy): {accuracy:.2f}")
print(f"Test seti üzerindeki F1 skoru: {f1:.2f}")

# Sınıflandırma raporunu yazdırma
print("Sınıflandırma Raporu:")
print(classification_report(y_test, y_pred_class, target_names=le.classes_, labels=np.unique(y_test)))

print("Unique classes in y_test:", np.unique(y_test))
print("Classes from LabelEncoder:", le.classes_)

# Karışıklık matrisini yazdırma
print("Karışıklık Matrisi:")
print(confusion_matrix(y_test, y_pred_class))


Model başarıyla eğitildi ve kaydedildi.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
Tahmin edilen kişi: aysu
Test seti üzerindeki doğruluk(accuracy): 0.78
Test seti üzerindeki F1 skoru: 0.81
Sınıflandırma Raporu:
              precision    recall  f1-score   support

        aysu       0.80      1.00      0.89         4
      dilara       1.00      0.50      0.67         2
       guzel       1.00      0.67      0.80         3

   micro avg       0.88      0.78      0.82         9
   macro avg       0.93      0.72      0.79         9
weighted avg       0.91      0.78      0.81         9

Unique classes in y_test: [1 2 3]
Classes from LabelEncoder: ['aysu' 'dilara' 'guzel' 'kadero']
Karışıklık Matrisi:
[[0 0 0 0]
 [0 4 0 0]
 [1 0 1 0]
 [0 1 0 2]]


c:\Python311\Lib\site-packages\sklearn\metrics\_classification.py:2687: UserWarning: labels size, 3, does not match size of target_names, 4
  warnings.warn(


KONUŞAN KİSİNİN KİM OLDUGUNU ANLIK OLARAK BULMAK

In [4]:
import sounddevice as sd
import wave

def ses_kaydet(output_file, sure=5, fs=44100):
    print("Kayıt başlıyor...")
    kayit = sd.rec(int(sure * fs), samplerate=fs, channels=1, dtype='int16')
    sd.wait()  # Kayıt tamamlanana kadar bekle
    print("Kayıt tamamlandı.")
    
    # Kaydı wav formatında kaydet
    with wave.open(output_file, 'wb') as wf:
        wf.setnchannels(1)  # Mono kanal
        wf.setsampwidth(2)  # 16-bit
        wf.setframerate(fs)
        wf.writeframes(kayit.tobytes())

# Mikrofondan kayıt yap ve tahmini gerçekleştir
kayıt_dosyasi = "mikrofon_kayit.wav"
ses_kaydet(kayıt_dosyasi)

tahmin = ses_tahmin_et("ses_tanima_modeli.keras", kayıt_dosyasi, le)
print(f"Mikrofondan kaydedilen sesin tahmini: {tahmin}")


Kayıt başlıyor...
Kayıt tamamlandı.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Mikrofondan kaydedilen sesin tahmini: aysu


TEXTE ÇEVİRME VE KELİME SAYISI BULMA

In [4]:
import speech_recognition as sr

# Tanıyıcıyı başlatın
recognizer = sr.Recognizer()

# Ses dosyasını tanıyın
audio_file = "mikrofon_kayit.wav"

with sr.AudioFile(audio_file) as source:
    audio = recognizer.record(source)

# Türkçe ses dosyasını metne çevirin
try:
    text = recognizer.recognize_google(audio, language="tr-TR")
    print("Metin:",text)
    
    # Metin üzerinde kelime sayısı hesaplama
    words =text.split()
    word_count = len(words)
    print(f"Kelime Sayısı: {word_count}")
    
except sr.UnknownValueError:
    print("Google Speech-to-Text, sesi anlayamadı.")
except sr.RequestError as e:
    print(f"Google Speech-to-Text servisi kullanılamıyor: {e}")
    



Metin: sevdiğin birinin beni anlamadığını hissetmek çok acı verici cevaplar
Kelime Sayısı: 9


Texte Çevirme, Duygu ve Konu Analizi

In [ ]:
import speech_recognition as sr
from pydub import AudioSegment
from transformers import pipeline
from deep_translator import GoogleTranslator

# MP3'ten WAV'a dönüştürme fonksiyonu
def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")

# Ses dosyasını metne çevirme fonksiyonu
def transcribe_audio(audio_file_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_file_path) as source:
        audio = recognizer.record(source)
    try:
        transcript = recognizer.recognize_google(audio, language="tr-TR")
        return transcript
    except sr.UnknownValueError:
        return "Could not understand audio"
    except sr.RequestError as e:
        return f"Could not request results; {e}"

# Metni Türkçeden İngilizceye çevirme fonksiyonu
def translate_to_english(text):
    translation = GoogleTranslator(source='tr', target='en').translate(text)
    return translation

# Zero-shot sınıflandırma için transformer modeli
topic_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
sentiment_analyzer = pipeline("text-classification", model="bhadresh-savani/bert-base-uncased-emotion", return_all_scores=True)

# Duygu Analizi Fonksiyonu
def analyze_emotion(text):
    results = sentiment_analyzer(text, truncation=True)
    emotions = {result['label']: result['score'] for result in results[0]}
    total = sum(emotions.values())
    percentages = {emotion: (score / total) * 100 for emotion, score in emotions.items()}
    return percentages

# Konu Analizi Fonksiyonu
def analyze_topic(text):
    topics = ["sport", "technology", "health", "art", "weather", "feelings"]
    result = topic_classifier(text, candidate_labels=topics)
    return result['labels'][0], result['scores'][0]

# Ana kod
if __name__ == "__main__":
    wav_file_path = "mikrofon_kayit.wav"  # Path to the converted WAV file

    # Ses dosyasını yazıya çevir
    transcript = transcribe_audio(wav_file_path)

    print("\nTranscript:")
    print(transcript)
    
    words =transcript.split()
    word_count = len(words)
    print(f"\nKelime Sayısı: {word_count}")

    # İngilizceye çevir
    translated_text = translate_to_english(transcript)
   
    # Duygu Analizi
    print("\nDuygu Analizi:")
    if translated_text:
        emotion_percentages = analyze_emotion(translated_text)
        for emotion, percentage in emotion_percentages.items():
            print(f"{emotion}: %{percentage:.2f}")
    else:
        print("Duygu analizi yapılamadı.")

    # Konu Analizi
    print("\nKonu Analizi:")
    topic, score = analyze_topic(translated_text)
    print(f"En uygun konu: {topic} (%{score*100:.2f})")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBartForSequenceClassification: ['model.decoder.version', 'model.encoder.version']
- This IS expected if you are initializing TFBartForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBartForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBartForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForSequenceClassification for predictions without further training.
Some weights of the PyTorch model were not used when initializing the TF 2.0 mode

Transcript:
sevdiğin birinin beni anlamadığını hissetmek çok acı verici cevaplar

Kelime Sayısı: 9

Duygu Analizi:
sadness: %99.13
joy: %0.58
love: %0.11
anger: %0.11
fear: %0.05
surprise: %0.02

Konu Analizi:
En uygun konu: feelings (%73.42)
